Let's try and find some bad labels! 

In [4]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)

df = pd.read_csv("data/annotations.csv")[['text', 'excitement']].drop_duplicates()

Let's train a model to help us. We'll just use a bag of words model here.

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
%%time 

model = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=1000))
model.fit(list(df['text']), df['excitement'])

CPU times: user 9.92 s, sys: 28.6 s, total: 38.5 s
Wall time: 3.76 s


Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('logisticregression', LogisticRegression(max_iter=1000))])

Let's now select predictions that don't match the label. 

In [7]:
df_pred = (df
           .assign(pred=lambda d: model.predict(d['text']), 
                   proba=lambda d: model.predict_proba(d['text'])[:, 1]))

n_found = df_pred.loc[lambda d: d['excitement'] != d['pred']].shape[0]
n_found, n_found/len(df)

(4298, 0.06939869534327972)

That's a lot of items! Let's do some sorting.

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

We'll first sort based on low confidence values. 

In [16]:
(df_pred
 .loc[lambda d: d['excitement'] != d['pred']]
 .sort_values("proba", ascending=True)
 .head(10))

,text,excitement,pred,proba
58295,What's your point? All these comments are from people who clearly had not seen it before. Only a couple said they had.,1,0,0.001899
51549,Did this a lot when I was a kid but never did it as bad as this,1,0,0.005372
161431,Damn... that’s another thing I didn’t even think about it. Probably the biggest reason why people shouldn’t steal food,1,0,0.007809
10903,"I’ll never understand people who freak out at that, just say they were born a different color.",1,0,0.008207
145924,True! He probably is just lonely. Thank you for the kind words :),1,0,0.008674
7676,Don’t get me wrong. I laugh so I don’t cry :’),1,0,0.012360
148797,I think [NAME] did such a classy exit! Hopefully people don’t give her too much hate. She comes from a good place.,1,0,0.012701
120189,It would've been pretty funny in May. Not anymore I'd say.,1,0,0.013742
1744,"The joke's on them, because I'm too ugly and petty to have a wife!",1,0,0.013989
195525,He’s been going to a restaurant for 40 years and forgot what it looked like and where it was.. uh huh.,1,0,0.014780


Next, we can sort based on high confidence values. 

In [19]:
(df_pred
 .loc[lambda d: d['excitement'] != d['pred']]
 .sort_values("proba", ascending=False)
 .head(10))

,text,excitement,pred,proba
60361,"omg I remember be exactly like this and being so excited, gahhhh I might even believe I’m more excited than you hahahah.",0,1,0.977509
106863,"[NAME] likes making us excited, but not *too* excited",0,1,0.964636
169932,Omg this is so amazing ! Keep up the awesome work and have a fantastic New Year !,0,1,0.951248
59062,"Happy cake day! Have a great day and year, cheers.",0,1,0.895536
82913,"That's an interesting way to do it. I might come off as a little crazy, but this would definitely be an interesting experiment.",0,1,0.886199
30527,Wow! That's awesome! Congratulations! Its wonderful to have hope of being off medications one day.,0,1,0.859720
75398,wow wow wow hold my beer!,0,1,0.857525
146901,I am inexplicably excited by [NAME]. I get so excited by how he curls passes,0,1,0.844191
25554,It's wonderful and gives me happy happy feels,0,1,0.836533
84338,Yeah cheers. Hoping it's gonna be better. Happy early new year to you too.,0,1,0.835726


Based on confidence, we get some other examples.

In [13]:
threshold = 0.01

(df_pred
 .loc[lambda d: d['proba'] > 0.5 - threshold]
 .loc[lambda d: d['proba'] < 0.5 + threshold]
 .head(5))

,text,excitement,pred,proba
3013,Imagining doing something this drastic all alone is just... I can't fathom the anxiety. Man.,0,0,0.490705
4637,"I did the same with a group of buddies. Nonetheless, Happy New Year!",1,0,0.491268
4761,Oh wait that's tonight?!?! Totally watching 😎,0,1,0.500616
5735,"“Oh hey, I think a second ago [NAME] just put a move on [NAME]! Oh boy I wonder how that was.”",0,1,0.507843
6341,This actually made me squeal like a little girl! Thats awesome!!!! Im so happy!,0,1,0.500256
